In [28]:
import glob
import os
from random import shuffle
import random

def pre_process_data(filepath):
    dataset = []
    dirs = []
    
    for dir in os.listdir(filepath):
        if os.path.isdir(filepath + dir):
            dirs.append(filepath + dir)
            
    for i, dir_path in enumerate(dirs):
        dir_name = dir_path.split('/')[-1]
        label_id = i
        print('label_id: {}, dir_name: {}'.format(label_id, dir_name))
        
        for filename in glob.glob(os.path.join(filepath, dir_name, dir_name + "*.txt")):
            with open(filename, 'r' ,encoding="utf-8") as f:
                #datasets hold sets of tuples such as (label, input text)
                dataset.append((label_id, f.read()))
                
    random.seed(1234)            
    shuffle(dataset)
    
    return dataset

In [29]:
path = "livedoor_data/text/"

dataset = pre_process_data(path)
dataset[1234]

label_id: 0, dir_name: dokujo-tsushin
label_id: 1, dir_name: it-life-hack
label_id: 2, dir_name: kaden-channel
label_id: 3, dir_name: livedoor-homme
label_id: 4, dir_name: movie-enter
label_id: 5, dir_name: peachy
label_id: 6, dir_name: smax
label_id: 7, dir_name: sports-watch
label_id: 8, dir_name: topic-news


(7,
 'http://news.livedoor.com/article/detail/6749212/\n2012-07-12T08:10:00+0900\n″ご立腹″の松木氏、関塚監督の采配について苦言呈す\n11日、国立競技場で行われた、キリンチャレンジカップ2012・U-23日本代表×U-23ニュージーランド代表の一戦は、日本代表が1点を先制し、終始優勢に試合を進めながらも、後半ロスタイムにイージーミスからボールを奪われ、まさかの失点。試合は1-1のドローに終わった。\n\n12日、フジテレビで放送された「めざましテレビ」では、サッカー解説者の松木安太郎氏が昨夜の一戦を振り返った。「かなりご立腹」と紹介された松木氏。「まず若さが目立ちましたね。せっかくいいカタチでボールを取って、いいカタチで攻めるんですけど、最後のもう一工夫がなかったり、シュートが遅れたり。バランスがすごくよくなかった」と語る。\n\nすると、松木氏は、関塚監督の采配についても言及。バックアップ・メンバーである、山崎亮平（磐田）や米本拓司（FC東京）が起用されたことに触れると、「ちょっと気になったのが、昨日、バックアップ・メンバーがもともとの18名を使う前に出てきたんですね。彼らがいけないということではなくて、必勝態勢のゲームでありながら、本大会のシミュレーションもしたいということを考えると、まずはバックアップメンバーでない選手から使うのが普通じゃないかな。その辺が最後の失点というか、心の緩みみに繋がっていったような気がします」と苦言を呈した。\n\n・宮本恒靖氏、U-23日本代表に「リスクを冒さないとダメ」\n・U-23日本代表、まさかのドロー。セルジオ氏は「まるで高校サッカーの決勝」\n')

In [4]:
len(dataset)

7367

In [8]:
import gensim

model = gensim.models.KeyedVectors.load_word2vec_format('model.vec', binary=False)
model.most_similar(positive=['日本'])

[('日本国内', 0.6301000118255615),
 ('アメリカ', 0.571485698223114),
 ('韓国', 0.5515788793563843),
 ('日本の歴史', 0.5300577878952026),
 ('日本国', 0.5054277181625366),
 ('日本の経済史', 0.5011717081069946),
 ('日本の魚', 0.4976142942905426),
 ('米国', 0.4971674680709839),
 ('中国', 0.49715349078178406),
 ('アメリカ合衆国', 0.49622151255607605)]

In [30]:
import MeCab

def tokenize_and_vectorize(dataset):
    vectorized_data = []
    tokenizer = MeCab.Tagger("-Owakati -d /var/lib/mecab/dic/mecab-ipadic-neologd")
    for sample in dataset:
        tokens_list = tokenizer.parse(sample[1]).split()
        each_sample_input_vecs = []
        for token in tokens_list:
            try:
                each_sample_input_vecs.append(model[token])
            except KeyError:
                pass
            
        vectorized_data.append(each_sample_input_vecs)
        
    return vectorized_data

In [31]:
def collect_labels(dataset):
    labels = []
    for sample in dataset:
        labels.append(sample[0])
    
    return labels

In [32]:
vectorized_inputs = tokenize_and_vectorize(dataset)
labels = collect_labels(dataset)

In [33]:
print('len(vectorized_inputs):', len(vectorized_inputs))
print('len(vectorized_inputs[0]):', len(vectorized_inputs[0]))
print('len(vectorized_inputs[0][0]):', len(vectorized_inputs[0][0]))
print('len(labels):', len(labels))

len(vectorized_inputs): 7367
len(vectorized_inputs[0]): 374
len(vectorized_inputs[0][0]): 300
len(labels): 7367


In [34]:
split_data = int(len(vectorized_inputs)* 0.8)

x_train = vectorized_inputs[:split_data]
x_test = vectorized_inputs[split_data:]
y_train= labels[:split_data]
y_test = labels[split_data:]

In [35]:
#To check the maximum input steps among the entire dataset
max = 0
for elem in vectorized_inputs:
    if len(elem) > max:
        max = len(elem)
        
print(max)

5760


In [36]:
#To check the minimus input steps among the entire dataset
min = 5760
for elem in vectorized_inputs:
    if len(elem) < min:
        min = len(elem)
        
print(min)

31


In [37]:
#To check the average input steps among the entire dataset
sum = 0
total_num = len(vectorized_inputs)
for elem in vectorized_inputs:
     sum += len(elem)
        
print(sum/total_num)

635.5489344373558


In [38]:
max_len = 512
embedding_dims = 300

In [39]:
from tqdm import tqdm

def pad_or_truncate_inputs(data, max_len):
    new_data = []
    pad_vec = []
    for _ in range(len(data[0][0])):
        pad_vec.append(0.0)
        
    print('len of pad_vec:', len(pad_vec))
        
    for sample in tqdm(data):
        if len(sample) >= max_len:
            tmp = sample[:max_len]
        else:
            tmp = sample
            num_of_pad_vecs_needed = max_len - len(sample)
            for _ in range(num_of_pad_vecs_needed):
                tmp.append(pad_vec)
                
        new_data.append(tmp)
        
    return new_data

In [40]:
import numpy as np

x_train = pad_or_truncate_inputs(x_train, max_len)
x_test = pad_or_truncate_inputs(x_test, max_len)
x_train = np.reshape(x_train, (len(x_train), max_len, embedding_dims))
x_test = np.reshape(x_test, (len(x_test), max_len, embedding_dims))

  0%|          | 0/5893 [00:00<?, ?it/s]

len of pad_vec: 300


100%|██████████| 1474/1474 [00:00<00:00, 52740.99it/s]


len of pad_vec: 300


In [46]:
print('x_train.shape:', x_train.shape)
print('x_test.shape:', x_test.shape)

x_train.shape: (5893, 500, 300)
x_test.shape: (1474, 500, 300)
